In [1]:
import sys

In [2]:
print(sys.executable)

/home/hbcho991/.conda/envs/R-Adapter/bin/python


In [ ]:
sys.path.append("PATH")

In [3]:
!python --version

Python 3.11.5


In [ ]:
!ln -s /home/dataset/imagenet1k/data ./datasets/data/ILSVRC2012

In [5]:
import sys

In [6]:
print(sys.path)

['/home/hbcho991/.conda/envs/R-Adapter/lib/python310.zip', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/lib-dynload', '', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/site-packages', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/site-packages/setuptools/_vendor']


In [ ]:
sys.path.insert(0,'/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!python datacreation_scripts/imagenet_csv_creator.py

In [ ]:
!pip install wandb

In [ ]:
!pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

In [ ]:
!pip install open_clip_torch

In [ ]:
!pip install wilds braceexpand webdataset h5py

In [ ]:
!pip install pycocotools

## full-tuning 실행

In [16]:
!torchrun --master_port 29400 --nproc_per_node 1 -m src.main     --train-dataset=ImageNet     --epochs=10     --lr=5e-4     --wd=0     --workers=4     --batch-size=32     --model=ViT-B/32     --eval-datasets=ImageNet,ImageNetV2     --template=openai_imagenet_template     --save=./checkpoints/     --data-location=./datasets/data/     --ft_data=./datasets/csv/imagenet_classID.csv     --adapter=16,16     --drop-path=0.2     --ema=0.999     --eval-scale=0.5     --grad-clip-norm=1     --ls=0.02     --mg=0.05     --eval_epoch=0     --csv-img-key=filepath     --csv-caption-key=title     --supervised-label-key=class     --exp_name=ImageNet/R-Adapter_full_finetune

/opt/anaconda3/2023.09-0/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
INFO:root:Running with a single process. Device cuda:4.
2024-10-28,21:17:10 | INFO | Running with a single process. Device cuda:4.
ViT-B/32
100%|███████████████████████████████████████| 354M/354M [00:09<00:00, 38.5MiB/s]
INFO:root:Namespace(data_location='./datasets/data/', eval_datasets=['ImageNet', 'ImageNetV2'], train_dataset='ImageNet', template='openai_imagenet_template', classnames='openai', alpha=[0.5], exp_name='ImageNet/R-Adapter_full_finetune', results_db=None, model='ViT-B/32', use_peft=True, lora=[-1, -1], adapter=[16, 16], drop_path=0.2, ema=0.999, bma=0, eval_scale=0.5, lock_image=False, lock_text=False, unlock_proj=False, unlock_ln=False, unlock_bias=False, unlock_cls=False, lock_classifier=False, batch_s

/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:138: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if args.precision == "amp" else None
INFO:root:Start epoch 0
2024-10-28,21:19:10 | INFO | Start epoch 0
/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:287: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
INFO:root:Train Epoch: 0 [     32/1281167 (0%)] Data (t): 0.000 Batch (t): 0.110, 290.690/s LR: 0.000002 Logit Scale: 100.000 Loss: 2.1933 (2.1933)
2024-10-28,21:19:17 | INFO | Train Epoch: 0 [     32/1281167 (0%)] Data (t): 0.000 Batch (t): 0.110, 290.690/s LR: 0.000002 Logit Scale: 100.000 Loss: 2.1933 (2.1933)
INFO:root:Train Epoch: 0 [  16032/1281167 (1%)] Data (t): 0.000 Batch (t): 0.056, 573.117/s LR: 0.000501 Logit Scale: 100.000 Loss: 1.3069 (1.7501)
2024-10-28,21:19:5

INFO:root:Train Epoch: 0 [ 400032/1281167 (31%)] Data (t): 0.000 Batch (t): 0.052, 612.676/s LR: 0.000500 Logit Scale: 100.000 Loss: 1.4288 (0.93254)
2024-10-28,21:31:14 | INFO | Train Epoch: 0 [ 400032/1281167 (31%)] Data (t): 0.000 Batch (t): 0.052, 612.676/s LR: 0.000500 Logit Scale: 100.000 Loss: 1.4288 (0.93254)
INFO:root:Train Epoch: 0 [ 416032/1281167 (32%)] Data (t): 0.000 Batch (t): 0.057, 557.832/s LR: 0.000500 Logit Scale: 100.000 Loss: 0.82140 (0.92842)
2024-10-28,21:31:43 | INFO | Train Epoch: 0 [ 416032/1281167 (32%)] Data (t): 0.000 Batch (t): 0.057, 557.832/s LR: 0.000500 Logit Scale: 100.000 Loss: 0.82140 (0.92842)
INFO:root:Train Epoch: 0 [ 432032/1281167 (34%)] Data (t): 0.000 Batch (t): 0.052, 620.646/s LR: 0.000500 Logit Scale: 100.000 Loss: 1.4547 (0.94722)
2024-10-28,21:32:10 | INFO | Train Epoch: 0 [ 432032/1281167 (34%)] Data (t): 0.000 Batch (t): 0.052, 620.646/s LR: 0.000500 Logit Scale: 100.000 Loss: 1.4547 (0.94722)
INFO:root:Train Epoch: 0 [ 448032/1281167

INFO:root:Train Epoch: 0 [ 816032/1281167 (64%)] Data (t): 0.000 Batch (t): 0.054, 594.871/s LR: 0.000496 Logit Scale: 100.000 Loss: 0.87734 (0.89451)
2024-10-28,21:43:42 | INFO | Train Epoch: 0 [ 816032/1281167 (64%)] Data (t): 0.000 Batch (t): 0.054, 594.871/s LR: 0.000496 Logit Scale: 100.000 Loss: 0.87734 (0.89451)
INFO:root:Train Epoch: 0 [ 832032/1281167 (65%)] Data (t): 0.000 Batch (t): 0.054, 590.622/s LR: 0.000496 Logit Scale: 100.000 Loss: 1.1380 (0.89910)
2024-10-28,21:44:10 | INFO | Train Epoch: 0 [ 832032/1281167 (65%)] Data (t): 0.000 Batch (t): 0.054, 590.622/s LR: 0.000496 Logit Scale: 100.000 Loss: 1.1380 (0.89910)
INFO:root:Train Epoch: 0 [ 848032/1281167 (66%)] Data (t): 0.000 Batch (t): 0.052, 613.138/s LR: 0.000496 Logit Scale: 100.000 Loss: 0.57093 (0.89303)
2024-10-28,21:44:38 | INFO | Train Epoch: 0 [ 848032/1281167 (66%)] Data (t): 0.000 Batch (t): 0.052, 613.138/s LR: 0.000496 Logit Scale: 100.000 Loss: 0.57093 (0.89303)
INFO:root:Train Epoch: 0 [ 864032/12811

INFO:root:Train Epoch: 0 [1232032/1281167 (96%)] Data (t): 0.000 Batch (t): 0.052, 610.541/s LR: 0.000490 Logit Scale: 100.000 Loss: 0.58050 (0.87725)
2024-10-28,21:56:01 | INFO | Train Epoch: 0 [1232032/1281167 (96%)] Data (t): 0.000 Batch (t): 0.052, 610.541/s LR: 0.000490 Logit Scale: 100.000 Loss: 0.58050 (0.87725)
INFO:root:Train Epoch: 0 [1248032/1281167 (97%)] Data (t): 0.000 Batch (t): 0.050, 636.399/s LR: 0.000490 Logit Scale: 100.000 Loss: 0.51101 (0.87261)
2024-10-28,21:56:29 | INFO | Train Epoch: 0 [1248032/1281167 (97%)] Data (t): 0.000 Batch (t): 0.050, 636.399/s LR: 0.000490 Logit Scale: 100.000 Loss: 0.51101 (0.87261)
INFO:root:Train Epoch: 0 [1264032/1281167 (99%)] Data (t): 0.000 Batch (t): 0.051, 623.228/s LR: 0.000489 Logit Scale: 100.000 Loss: 0.73034 (0.87083)
2024-10-28,21:56:58 | INFO | Train Epoch: 0 [1264032/1281167 (99%)] Data (t): 0.000 Batch (t): 0.051, 623.228/s LR: 0.000489 Logit Scale: 100.000 Loss: 0.73034 (0.87083)
INFO:root:Train Epoch: 0 [1280032/128

In [9]:
!nvidia-smi

Mon Oct 28 20:45:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:17:00.0 Off |                  Off |
| 43%   54C    P2             183W / 450W |  16651MiB / 24564MiB |     36%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## few-shot 실행

In [ ]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.few_shot     --train-dataset=ImageNet     --epochs=50     --lr=5e-4     --wd=0     --workers=4     --batch-size=32     --model=ViT-B/16     --eval-datasets=ImageNet,ImageNetV2,ImageNetR,ImageNetSketch,ImageNetA     --template=openai_imagenet_template     --save=./checkpoints/     --data-location=./datasets/data/     --ft_data=./datasets/csv/imagenet_classID.csv     --adapter=4,4     --drop-path=0.2     --ema=0.999     --eval-scale=0.5     --grad-clip-norm=1     --ls=0.02     --mg=0.05     --csv-img-key=filepath     --csv-caption-key=title     --supervised-label-key=class     --warmup_length=0     --k=16     --exp_name=ImageNet/R-Adapter_few_shot_16shot


## base-to-novel 실행

In [ ]:
!ln -s /home/dataset/flowers ./datasets/data/flowers102

In [ ]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.main_few_shot_novel         --train-dataset=flowers102        --epochs=101         --lr=5e-4         --wd=0         --workers=4         --batch-size=32         --model=ViT-B/16         --template=openai_imagenet_template         --save=./checkpoints/         --data-location=./datasets/data/         --adapter=4,4         --drop-path=0.2         --ema=0.9         --eval-scale=0.5         --grad-clip-norm=1         --ls=0         --mg=0.1         --csv-img-key=filepath         --csv-caption-key=title         --supervised-label-key=class         --warmup_length=500         --k=16         --save=False         --exp_name=${dataset}/R-Adapter_few_shot_16shot